# Import Modules

In [1]:
%load_ext autoreload
%autoreload
%pylab inline
import os
import glob
import json
import pandas as pd
import pickle

#from IPython import embed
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 100)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/tmp/ipykernel_1446478/197202962.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
from helper_functions import *

# Read and Process Excel Sheets (Organized by Fuel Type and Labeled as Burnable or Unburnable)

In [3]:
data_base_path = 'raw_data'

### Read the processed excel sheet with alerts and confusion matrix

In [4]:
#excel_processed = pd.ExcelFile(os.path.join(data_base_path, 'GroundTruth_FuelTypes13_Labeled.xlsx'))
excel_processed = pd.ExcelFile(os.path.join(data_base_path, 'GroundTruth_FuelTypes05_Labeled.xlsx'))
#excel_processed = pd.ExcelFile(os.path.join(data_base_path, 'GroundTruth_FuelMoisture.xlsx'))

In [5]:
sheet_to_df_map_read = {}

In [8]:
#features_to_use = ['feature01', 'feature02', 'feature03', 'feature04','feature05', 'feature06', 'feature07']
features_to_use = ['DN1', 'DN2', 'DN3', 'DN4']
#features_to_use = ['WS_t1', 'WS_t2', 'WS_tn', 'T_t1', 'T_t2', 'T_tn', 'QoI_t1', 'QoI_t2', 'QoI_tn']
#features_to_use = ['WS_t1', 'WS_t2', 'T_t1', 'T_t2']

In [9]:
for sheet in excel_processed.sheet_names:
    df_read = pd.DataFrame(pd.read_excel(excel_processed, sheet))
    if sheet == 'Summary':
        sheet_to_df_map_read[sheet] = df_read
    else:            
        sheet_to_df_map_read[sheet] = df_read[['SheetName'] + features_to_use + ['Label']]
del sheet_to_df_map_read['Summary']

In [10]:
sheet_to_df_map_read.keys()
#sheet_to_df_map_read['Summary']

dict_keys(['Fuel01', 'Fuel02', 'Fuel03', 'Fuel04', 'Fuel05'])

In [12]:
#sheet_to_df_map_read['FuelMoisture'][features_to_use + ['Label']]

In [13]:
sheet_to_df_map_read['Fuel05'].head(5)

,SheetName,DN1,DN2,DN3,DN4,Label
0,Fuel05,11.102183,4.29008,9.37405,-77.694054,Fuel05_Unburnable
1,Fuel05,11.876525,10.73280,3.78626,3.165404,Fuel05_Unburnable
2,Fuel05,7.873579,8.04580,14.06110,-59.000118,Fuel05_Unburnable
3,Fuel05,7.805673,6.41221,7.64885,-70.938026,Fuel05_Unburnable
4,Fuel05,10.082099,8.39695,19.26720,69.736328,Fuel05_Unburnable


### Define features and labels

In [ ]:
#features = ['feature01', 'feature02', 'feature03', 'feature04','feature05', 'feature06', 'feature07']
features = ['DN1', 'DN2', 'DN3', 'DN4']
labels = ['Label', 'NewLabel', 'SheetName']
cols = labels + features

### Re-Label sheets and combine and prune them

In [ ]:
sheet_to_df_map = re_label_data(sheet_to_df_map_read)
df_combined = combine_df_sheets(sheet_to_df_map, sorted(sheet_to_df_map.keys()), cols)
df_combined_orig = df_combined
len(df_combined)

In [ ]:
len(df_combined)
#df_combined.keys()

# SVM Stuff

### Scikit-Learn Imports

In [ ]:
import matplotlib.pyplot as plt
from sklearn.svm import SVC
#from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, average_precision_score
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_recall_curve, classification_report
from time import time

### Work with Multiple Features of Interest

In [ ]:
#features = ['feature01', 'feature02', 'feature03', 'feature04','feature05', 'feature06', 'feature07']
features = ['DN1', 'DN2', 'DN3', 'DN4']

#### Get Features and Labels for Training and Testing

In [ ]:
X, y = get_features_labels(df_combined, features_to_use)
features_train, features_test, labels_train, labels_test = train_test_split(X, y, test_size=0.2, random_state=42)
sheets_train, sheets_test, labels_train, labels_test = split_labels_sheets(labels_train, labels_test)
len(features_train), len(features_test), len(labels_train), len(labels_test)

#### Create and Train the Model

In [ ]:
########################## SVM #################################
# Training Time is significantly more with large values of C
clf = SVC(kernel="linear", class_weight = "balanced") #class_weight
# Try RBF etc., gamma term, epsilon, C
# Grid search to optimize parameters
# Semantic segmentation of pixels
# Try Random Forest
# Google earth Engine for satellite images in addition to QGIS

### fit the classifier on the training features and labels
t0 = time()
clf.fit(features_train, labels_train)
print ("Training Time:", round(time()-t0, 3), "s")

#### Save the Model to a File

In [ ]:
# save the model to disk
'svm_N1205_T080_C1 --> svm_N(number of total data)_T(percentage for training)_C(model parameter in model constructor, default 1)'
svm_model_file = 'models_trained/svm_N1205_T080_C1'
pickle.dump(clf, open(svm_model_file, 'wb'))

#### Load the Model from a File (A template...use specific model for specific pupose)

In [ ]:
# load the model from disk

## Predict on the Train Data itself using SVM

In [ ]:
clf = pickle.load(open('models_trained/svm_N1205_T080_C1', 'rb'))
t1 = time()
labels_svm_pred = clf.predict(features_train)
print ("Prediction Time:", round(time()-t1, 3), "s")
#print ('labels_svm_pred: {}'.format(labels_svm_pred))

accuracy = accuracy_score(labels_svm_pred, labels_train)
print('Accuracy Score: {}'.format(accuracy))

conf_mat = confusion_matrix(labels_train, labels_svm_pred, labels = [1, 0])
print('Confusion Matrix: \n{}'.format(conf_mat))

print('Classification Report: \n')
print(classification_report(labels_train, labels_svm_pred, labels=[1, 0]))

average_precision = average_precision_score(labels_train, labels_svm_pred)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

## Predict on the Test Data using SVM

In [ ]:
### use the trained classifier to predict labels for the test features using SVM predict
clf = pickle.load(open('models_trained/svm_N1205_T080_C1', 'rb'))
t1 = time()
labels_svm_pred = clf.predict(features_test)
print ("Prediction Time:", round(time()-t1, 3), "s")
#print ('labels_svm_pred: {}'.format(labels_svm_pred))

accuracy = accuracy_score(labels_svm_pred, labels_test)
print('Accuracy Score: {}'.format(accuracy))

conf_mat = confusion_matrix(labels_test, labels_svm_pred, labels = [1, 0])
print('Confusion Matrix: \n{}'.format(conf_mat))

print('Classification Report: \n')
print(classification_report(labels_test, labels_svm_pred, labels=[1, 0]))

average_precision = average_precision_score(labels_test, labels_svm_pred)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

"""
precision, recall, thresholds = precision_recall_curve(labels_test, pred)
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
"""

## Work with Feature Pairs

In [ ]:
features = features_to_use
for i in range(len(features)):
    for j in range(i+1, len(features)):
        #print('i: {}, j: {}'.format(i, j))
        v1 = features[i]
        v2 = features[j]
        print('---- v1: {}, v2: {} ------'.format(v1, v2))
        X, y = get_features_labels(df_combined, [v1, v2])
        features_train, features_test, labels_train, labels_test = train_test_split(X, y, test_size=0.1, random_state=42)
        sheets_train, sheets_test, labels_train, labels_test = split_labels_sheets(labels_train, labels_test)
        
        #len(features_train), len(features_test)
        clf = SVC(kernel="rbf", class_weight = "balanced") #class_weight
        
        ### fit the classifier on the training features and labels
        t0 = time()
        clf.fit(features_train, labels_train)
        print ("Training Time:", round(time()-t0, 3), "s")

        ### use the trained classifier to predict labels for the test features
        t1 = time()
        pred = clf.predict(features_test)
        #print ("Prediction Time:", round(time()-t1, 3), "s")
        #print ('pred: {}'.format(pred))

        accuracy = accuracy_score(pred, labels_test)
        #print('Accuracy Score: {}'.format(accuracy))
        
        plot_scatter_svm (clf, features_test, labels_test, v1, v2, accuracy, plot_base_dir = 'plots', scatter_dir = 'SVM')
        plot_scatter_arr (features_test, labels_test, v1, v2, accuracy, plot_base_dir = 'plots', scatter_dir = 'scatter/test_data')

## Plot Statistics/ Histogram

In [ ]:
col_for_hist = features_to_use

In [ ]:
plot_hist_dfmap_combined(sheet_to_df_map, ['Fuel01', 'Fuel02', 'Fuel03','Fuel04'],
                         col_for_hist, 'histogram/Burnable', plot_base_dir = 'plots')

In [ ]:
plot_hist_dfmap_combined(sheet_to_df_map, ['Fuel05'],
                         col_for_hist, 'histogram/Unburnable', plot_base_dir = 'plots')

## Create Scatter Plots

In [ ]:
burnable_sheets = ['Fuel01', 'Fuel02', 'Fuel03','Fuel04']
unburnable_sheets = ['Fuel05']
v_list = features_to_use
for i in range(len(v_list)):
    for j in range(i+1, len(v_list)):
        #print('i: {}, j: {}'.format(i, j))
        plot_scatter_df(sheet_to_df_map, burnable_sheets, unburnable_sheets, v_list[i], v_list[j], 'plots', 'scatter/all_data')